In [1]:
%matplotlib inline

In [2]:
import gym
import numpy as np
import random as pr
import matplotlib.pyplot as plt

In [3]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

env = gym.make("FrozenLake-v0")

learning_rate = 0.1
input_size = env.observation_space.n
output_size = env.action_space.n
dis = .99


# input: 16, output: 4, bias: 4 ==> 68
model = tf.keras.Sequential([
    Dense(output_size, input_shape=[input_size],
         kernel_initializer=tf.random_uniform_initializer(minval=0,maxval=0.01)) 
])
#model.compile(optimizer="sgd", loss='mse')
opt = tf.keras.optimizers.SGD(learning_rate=learning_rate)
model.compile(optimizer=opt, loss='mse')
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 68        
Total params: 68
Trainable params: 68
Non-trainable params: 0
_________________________________________________________________


In [4]:
def one_hot(x):
    return np.identity(16)[x:x+1]

## GPU version

In [13]:
num_episodes = 2000

rand_numbers = np.random.rand(num_episodes)
rand_numbers_idx = len(rand_numbers)
def get_rand():
    global rand_numbers, rand_numbers_idx
    if rand_numbers_idx == 0:
        rand_numbers = np.random.rand(num_episodes)
        rand_numbers_idx = len(rand_numbers)
    rand_numbers_idx -= 1
    return rand_numbers[i]

with tf.device('/GPU:0'):
    rList=[]

    for i in range(num_episodes):
        s = env.reset()
        e = 1.0 / ((i/50)+10)
        rAll = 0
        done = False
        local_loss = []

        Qs_all = []
        x_all = []
        while not done:
            Qs = model.predict(one_hot(s))
            if get_rand() < e:
                a = env.action_space.sample()
            else:
                a = np.argmax(Qs)

            s1, reward, done, _ = env.step(a)
            if done:
                Qs[0, a] = reward
            else:
                Qs1 = model.predict(one_hot(s1))
                Qs[0, a] = reward + dis*np.max(Qs1)

            Qs_all.append(Qs[0])
            x_all.append(one_hot(s)[0])
            # model.fit(x=one_hot(s), y=Qs, verbose=0)

            rAll += reward
            s= s1
        
        print(f"Episode: {i}")
        #print(np.array(x_all))
        #print(np.array(Qs_all))
        model.fit(x=np.array(x_all), y=np.array(Qs_all)) 
  
        #if i % 10 == 0:
        #    print(f"Episode: {i}")
        #    print(rAll)
        rList.append(rAll)
        
print("Percent of successful episode: "+str(sum(rList)/num_episodes)+"%")
plt.bar(range(len(rList)), rList, color='blue')
plt.show()

Episode: 0
1/1 [==============================] - 0s 42ms/step - loss: 6.1600e-07
Episode: 1
1/1 [==============================] - 0s 23ms/step - loss: 1.7955e-06
Episode: 2
1/1 [==============================] - 0s 40ms/step - loss: 1.4496e-06
Episode: 3
1/1 [==============================] - 0s 19ms/step - loss: 9.7999e-07
Episode: 4
1/1 [==============================] - 0s 27ms/step - loss: 7.3190e-07
Episode: 5
1/1 [==============================] - 0s 9ms/step - loss: 4.4369e-07
Episode: 6
2/2 [==============================] - 0s 40ms/step - loss: 4.7094e-07
Episode: 7
1/1 [==============================] - 0s 20ms/step - loss: 6.2571e-07
Episode: 8
1/1 [==============================] - 0s 17ms/step - loss: 1.1322e-06
Episode: 9
1/1 [==============================] - 0s 18ms/step - loss: 7.4989e-07


## Non GPU version

In [ ]:
num_episodes = 2000

rand_numbers = np.random.rand(num_episodes)
rand_numbers_idx = len(rand_numbers)
def get_rand():
    global rand_numbers, rand_numbers_idx
    if rand_numbers_idx == 0:
        rand_numbers = np.random.rand(num_episodes)
        rand_numbers_idx = len(rand_numbers)
    rand_numbers_idx -= 1
    return rand_numbers[i]

rList=[]
for i in range(num_episodes):
    s = env.reset()
    e = 1.0 / ((i/50)+10)
    rAll = 0
    done = False
    local_loss = []

    Qs_all = []
    x_all = []
    while not done:
        Qs = model.predict(one_hot(s))
        if get_rand() < e:
            a = env.action_space.sample()
        else:
            a = np.argmax(Qs)

        s1, reward, done, _ = env.step(a)
        if done:
            Qs[0, a] = reward
        else:
            Qs1 = model.predict(one_hot(s1))
            Qs[0, a] = reward + dis*np.max(Qs1)

        Qs_all.append(Qs[0])
        x_all.append(one_hot(s)[0])
        # model.fit(x=one_hot(s), y=Qs, verbose=0)

        rAll += reward
        s= s1

    print(f"Episode: {i}")
    #print(np.array(x_all))
    #print(np.array(Qs_all))
    model.fit(x=np.array(x_all), y=np.array(Qs_all)) 

    #if i % 10 == 0:
    #    print(f"Episode: {i}")
    #    print(rAll)
    rList.append(rAll)
        
print("Percent of successful episode: "+str(sum(rList)/num_episodes)+"%")
plt.bar(range(len(rList)), rList, color='blue')
plt.show()

Episode: 0
1/1 [==============================] - 0s 45ms/step - loss: 6.0577e-07
Episode: 1
1/1 [==============================] - 0s 16ms/step - loss: 3.3589e-07
Episode: 2
1/1 [==============================] - 0s 13ms/step - loss: 8.9118e-07
Episode: 3
1/1 [==============================] - 0s 12ms/step - loss: 6.6041e-07
Episode: 4
1/1 [==============================] - 0s 23ms/step - loss: 2.2064e-07
Episode: 5
1/1 [==============================] - 0s 11ms/step - loss: 6.1599e-07
Episode: 6
1/1 [==============================] - 0s 9ms/step - loss: 6.4120e-07
Episode: 7
1/1 [==============================] - 0s 25ms/step - loss: 5.4365e-07
Episode: 8
1/1 [==============================] - 0s 26ms/step - loss: 4.4937e-07
Episode: 9
1/1 [==============================] - 0s 5ms/step - loss: 1.0503e-07
Episode: 10
1/1 [==============================] - 0s 21ms/step - loss: 4.7031e-07
Episode: 11
1/1 [==============================] - 0s 13ms/step - loss: 4.3913e-07
Episode: 12
1/1 